## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup - most of this from VacationPy exercise
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.9667,102.5000,-26.05,100,100,4.47,overcast clouds
1,1,Saint-Philippe,RE,-21.3585,55.7679,80.19,89,100,11.86,light rain
2,2,Castro,BR,-24.7911,-50.0119,69.19,67,28,10.56,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,57.31,54,75,31.07,broken clouds
4,4,Barrow,US,71.2906,-156.7887,-25.58,68,0,4.61,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint-Philippe,RE,-21.3585,55.7679,80.19,89,100,11.86,light rain
6,6,Touros,BR,-5.1989,-35.4608,83.57,69,71,11.52,moderate rain
8,8,Ongandjera,NaN,-17.8833,15.0667,80.13,51,95,11.03,light rain
10,10,Karratha,AU,-20.7377,116.8463,85.60,69,100,19.82,overcast clouds
14,14,Bambous Virieux,MU,-20.3428,57.7575,80.85,94,75,6.91,broken clouds
15,15,Rikitea,PF,-23.1203,-134.9692,79.20,74,4,16.31,light rain
18,18,Inhambane,MZ,-23.8650,35.3833,76.98,69,40,6.91,scattered clouds
20,20,Hilo,US,19.7297,-155.0900,81.63,75,20,6.91,few clouds
22,22,Umm Kaddadah,SD,13.6017,26.6876,79.27,8,5,12.48,clear sky
25,25,Porto Novo,BJ,6.4965,2.6036,85.82,75,100,6.55,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                221
City                   221
Country                221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Create a Dataframe from a CSV
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()   

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, current description and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,
6,Touros,BR,83.57,moderate rain,-5.1989,-35.4608,
10,Karratha,AU,85.60,overcast clouds,-20.7377,116.8463,
14,Bambous Virieux,MU,80.85,broken clouds,-20.3428,57.7575,
15,Rikitea,PF,79.20,light rain,-23.1203,-134.9692,
18,Inhambane,MZ,76.98,scattered clouds,-23.8650,35.3833,
20,Hilo,US,81.63,few clouds,19.7297,-155.0900,
22,Umm Kaddadah,SD,79.27,clear sky,13.6017,26.6876,
25,Porto Novo,BJ,85.82,light rain,6.4965,2.6036,
30,Mohgaon,IN,83.79,broken clouds,21.6500,78.7167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Touros,BR,83.57,moderate rain,-5.1989,-35.4608,INN NEW HORIZON
10,Karratha,AU,85.60,overcast clouds,-20.7377,116.8463,ibis Styles Karratha
14,Bambous Virieux,MU,80.85,broken clouds,-20.3428,57.7575,Casa Tia Villa
15,Rikitea,PF,79.20,light rain,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """ 
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
#<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5
                )
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))